In [ ]:
!pip install python-telegram-bot

     |████████████████████████████████| 450kB 7.9MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 


In [1]:
import logging
#from typing import Tuple, Dict, Any

from telegram import InlineKeyboardMarkup, InlineKeyboardButton, Update, ReplyKeyboardMarkup
from telegram.ext import (
    Updater,
    CommandHandler,
    MessageHandler,
    Filters,
    ConversationHandler,
    CallbackQueryHandler,
    CallbackContext,
)

In [13]:
# State definitions for top level conversation
CHOOSING_SCENARIO, CHOOSING_USER_ID, CHOOSING_GR_ID, CHOOSING_CUSTOM = map(chr, range(4))
# State definitions for second level conversation
SAVING_USER_ID, SELECTING_GENDER = map(chr, range(4, 6))
# State definitions for descriptions conversation
SELECTING_ENGINE, TYPING_USER, TYPING_GOODREADS = map(chr, range(6, 9))
# Meta states
STOPPING, SHOWING = map(chr, range(9, 11))
# Shortcut for ConversationHandler.END
END = ConversationHandler.END

In [3]:
from telegram.ext import Updater
from getpass import getpass
updater = Updater(token=getpass(), use_context=True)

··········


In [4]:
dispatcher = updater.dispatcher

In [5]:
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)

In [6]:
def start(update, context):
    text = "Choose scenario"

    buttons = [[
                'User Id',
                'GoodReads Id',
                #'Custom Setup'
                ]]
    keyboard = ReplyKeyboardMarkup(buttons, one_time_keyboard=True)

    update.message.reply_text(text=text, reply_markup=keyboard)

    return CHOOSING_SCENARIO

In [7]:
def recommend(update, context):
  text = "Choose recommender engine"

  buttons = [['KNN', 'SVD']]

  keyboard = ReplyKeyboardMarkup(buttons, one_time_keyboard=True)
  update.message.reply_text(text=text, reply_markup=keyboard)

  return SELECTING_ENGINE

In [8]:
def ask_user_id(update, context):
  text = 'Type a number between 1 and 10000'
  update.message.reply_text(text=text)
  return TYPING_USER

def ask_user_id_again(update, context):
  text = 'Wrong input. Type a number between 1 and 10000'
  update.message.reply_text(text=text)
  return TYPING_USER

def save_user_id(update, context):
  try:
    context.user_data['user_id'] = int(update.message.text)
    return recommend(update, context)
  except ValueError:
    return ask_user_id_again(update, context)

In [9]:
def ask_goodreads_id(update, context):
  text = 'Type your GoodReads Id'
  update.message.reply_text(text=text)
  return TYPING_GOODREADS

def ask_goodreads_id_again(update, context):
  text = 'User doesnt exist or hasnt rated anything. Please try again or choose different scenario by calling /start'
  update.message.reply_text(text=text)
  return TYPING_GOODREADS

def save_goodreads_id(update, context):
  try:
    context.user_data['goodreads_id'] = int(update.message.text)
    return recommend(update, context)
  except ValueError:
    return ask_goodreads_id_again(update, context)

In [10]:
def rec_knn(update, context):
  update.message.reply_text(text='some recommendations for ' + str(context.user_data['user_id']))

In [14]:
dispatcher.add_handler(ConversationHandler(
  entry_points = [CommandHandler('start', start)],
  states = {
      CHOOSING_SCENARIO: [MessageHandler(Filters.regex('^User Id$'), ask_user_id),
                          MessageHandler(Filters.regex('^GoodReads Id$'), ask_goodreads_id)],
      TYPING_USER: [MessageHandler(Filters.text, save_user_id)],
      TYPING_GOODREADS: [MessageHandler(Filters.text, save_goodreads_id)],
      SELECTING_ENGINE: [MessageHandler(Filters.regex('^KNN$'), rec_knn)]

            },
  fallbacks = [CommandHandler('start', start)]))

In [15]:
updater.start_polling()

2021-04-19 14:09:01,302 - apscheduler.scheduler - INFO - Scheduler started


In [ ]:
updater.stop()

2021-04-19 13:14:17,017 - apscheduler.scheduler - INFO - Scheduler has been shut down
